## Tensors

In [1]:
import torch
import numpy as np

In [50]:
a=torch.FloatTensor(2,3)
b=torch.FloatTensor([[1,2,3],[3,2,1]])
c=torch.tensor(np.zeros(shape=(2,3)),dtype=torch.float32)
d=torch.tensor([1,2,3])
s=d.sum()

## GPU tensors

In [53]:
ag=torch.cuda.FloatTensor(2,3)
bg=torch.cuda.FloatTensor([[1,2,3],[3,2,1]])
cg=torch.cuda.FloatTensor(np.zeros(shape=(2,3)))
dg=torch.cuda.IntTensor([1,2,3])
sg=dg.sum()
eg=torch.cuda.IntTensor(1)

## CPU and GPU/CUDA transfer

In [64]:
ac=a.to(torch.device("cuda:0")).to(torch.device("cpu"))
bc=b.to("cuda:0").to("cpu")
cc=c.cuda(device=0).cpu()

## Gradient

In [94]:
v1=torch.tensor([1.,1.],requires_grad=True)
v2=torch.tensor([2.,2.])
v_sum=v1+v2
v_res=(v_sum*2).sum()
print(v_sum.is_leaf,v_res.is_leaf,v1.is_leaf,v2.is_leaf)
print(v_sum.requires_grad,v_res.requires_grad,v1.requires_grad,v2.requires_grad)

False False True True
True True True False


In [95]:
v_res.backward()
v1.grad,v2.grad

(tensor([2., 2.]), None)

## NN building blocks

In [181]:
from torch import nn
l=nn.Linear(2,5)
v=torch.FloatTensor([1,2])
l.zero_grad()
l(v)
l.state_dict()['weight'], l.state_dict()["bias"]

(tensor([[ 0.5137, -0.1095],
         [ 0.1600,  0.6351],
         [-0.0566,  0.5130],
         [ 0.4563, -0.4380],
         [ 0.4440,  0.5241]]),
 tensor([-0.4219,  0.6126,  0.3535, -0.1582,  0.6087]))

In [137]:
s=nn.Sequential(
    nn.Linear(2,5),
    nn.ReLU(),
    nn.Linear(5,20),
    nn.ReLU(),
    nn.Linear(20,10),
    nn.Dropout(p=0.3),
    nn.Softmax(dim=1))
s

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
  (5): Dropout(p=0.3)
  (6): Softmax()
)

In [140]:
s(torch.FloatTensor([[1,2]]))

tensor([[0.0947, 0.0869, 0.0947, 0.0945, 0.0831, 0.1181, 0.0866, 0.0947, 0.1450,
         0.1018]], grad_fn=<SoftmaxBackward>)

In [167]:
class OurModule(nn.Module):
    
    def __init__(self,num_inputs,num_classes,dropout_prob=0.3):
        super(OurModule,self).__init__()
        self.pipe=nn.Sequential(
            nn.Linear(num_inputs,5),
            nn.ReLU(),
            nn.Linear(5,20),
            nn.ReLU(),
            nn.Linear(20,num_classes),
            nn.Dropout(p=dropout_prob),
            nn.Softmax(dim=-1))
        
    def forward(self,x):
        return self.pipe(x)
    
net=OurModule(num_inputs=2,num_classes=3)
v=torch.FloatTensor([2,3])
out=net(v)
net, out    

(OurModule(
   (pipe): Sequential(
     (0): Linear(in_features=2, out_features=5, bias=True)
     (1): ReLU()
     (2): Linear(in_features=5, out_features=20, bias=True)
     (3): ReLU()
     (4): Linear(in_features=20, out_features=3, bias=True)
     (5): Dropout(p=0.3)
     (6): Softmax()
   )
 ), tensor([0.2467, 0.4328, 0.3205], grad_fn=<SoftmaxBackward>))

In [191]:
BigModule=nn.Sequential(
    nn.Linear(2,3),
    nn.ReLU(),
    OurModule(3,20),
    nn.ReLU(),
    nn.Softmax(dim=-1))
BigModule.cuda(0)(torch.cuda.FloatTensor([1,2]))

tensor([0.0502, 0.0500, 0.0496, 0.0504, 0.0496, 0.0500, 0.0500, 0.0510, 0.0500,
        0.0502, 0.0492, 0.0500, 0.0512, 0.0494, 0.0502, 0.0498, 0.0498, 0.0500,
        0.0495, 0.0500], device='cuda:0', grad_fn=<SoftmaxBackward>)

## Monitoring with TensorBoard

In [198]:
import math
from tensorboardX import SummaryWriter

writer=SummaryWriter(log_dir="test_run")
funcs={"sin":math.sin,"cos":math.cos,"tan":math.tan}

for angle in range(-360,360):
    angle_rad=angle*math.pi/180
    for name, fun in funcs.items():
        val=fun(angle_rad)
        writer.add_scalar(name,val,angle)
writer.close()